In [2]:
import os
import mido
import utils ##
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

Datos preprocesados:
https://drive.google.com/drive/folders/1jzfK-VM_Qb76K31Y7iMpsi1JeQ7QqU5W?usp=sharing

In [5]:
df_midis_info = pd.read_csv('datos_procesados/midis_info.csv')
df_midis_notas = pd.read_csv('datos_procesados/midis_notas.csv', index_col=0)

# MIDIS

In [4]:
df_midis_info[df_midis_info.tempo_ms == df_midis_info.tempo_ms.min()]

,key,Genero,PersonID,MusicID,SegmentID,RepetitionID,MetaID,tempo_ms,LimInf_Nota,LimSup_Nota,LimInf_Freq,LimSupFreq,LimInf_Cifrado,LimSup_Cifrado
666,F01_0184_0001_1_D,F,F01,184,1,1,D,384615,55,64,195.997718,329.627557,G3,E4
667,F01_0184_0001_2_D,F,F01,184,1,2,D,384615,55,64,195.997718,329.627557,G3,E4
668,F01_0184_0002_1_D,F,F01,184,2,1,D,384615,55,65,195.997718,349.228231,G3,F4
669,F01_0184_0002_2_D,F,F01,184,2,2,D,384615,55,65,195.997718,349.228231,G3,F4
670,F01_0184_0003_1_D,F,F01,184,3,1,D,384615,55,69,195.997718,440.000000,G3,A4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12575,M04_0184_0002_2_D,M,M04,184,2,2,D,384615,55,65,195.997718,349.228231,G3,F4
12576,M04_0184_0003_1_D,M,M04,184,3,1,D,384615,55,69,195.997718,440.000000,G3,A4
12577,M04_0184_0003_2_D,M,M04,184,3,2,D,384615,55,69,195.997718,440.000000,G3,A4
12578,M04_0184_0004_1_D,M,M04,184,4,1,D,384615,55,69,195.997718,440.000000,G3,A4


In [7]:
import midi
top_midi_fr = 2637.02 #E7
1 / (top_midi_fr * 2)

0.00018960796656832335

La frecuencia de muestreo es $44,100$ por lo que las ventanas de muestreo deben tener una frecuencia de al menos $88,200$ lo que es equivalente a $11.33\mu s$

In [10]:
# Ruta a la carpeta que contiene los archivos MIDI
path = 'Datos/MIDIs/test/'
# Duración de la ventana L en microsegundos
L = 11  # 189 micro 

# Procesar los archivos MIDI
sequences = midi.process_midi_files(path, L)

In [8]:
pd.Series(sequences['F04_0300_0001_1_D.mid']).value_counts()

1    1110377
0         33
2         33
Name: count, dtype: int64

In [15]:
mido.MidiFile('Datos/MIDIs/midi_data/F01_0001_0001_1_D.mid')

MidiFile(type=1, ticks_per_beat=1024, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=1024)]),
  MidiTrack([
    MetaMessage('track_name', name='é\x92¢ç\x90´, Piano', time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('pitchwheel', channel=0, pitch=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=60, velocity=80, time=0),
    Message('note_off', channel=0, note=60, velocity=0, time=1024),
    Message('note_on', channel=0, note=62, velocity=80, time=0),
    Message('note_off', channel=0, note=62, velocity=0, time=1024),
    Message('note_on', channel=0, note=64, velocity=80, time=0),
    Message('note

In [14]:
sequences['F01_0001_0001_1_D.mid']

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


# Tarareos


In [3]:
import tarareos

In [4]:
path = 'datos/Tarareos/test/'
# Duración de la ventana en milisegundos
L = 55  # por ejemplo, 500 milisegundos

# Procesar los archivos de audio y generar los espectrogramas
tarareos.process_audio_files(path, L)

# Otros

In [17]:
midi = mido.MidiFile('Datos/MIDIs/midi_data/F04_0300_0001_1_D.mid')
midi

MidiFile(type=1, ticks_per_beat=1024, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=857143, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=1024)]),
  MidiTrack([
    MetaMessage('track_name', name='é\x92¢ç\x90´, Piano', time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('pitchwheel', channel=0, pitch=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=65, velocity=80, time=0),
    Message('note_off', channel=0, note=65, velocity=0, time=768),
    Message('note_on', channel=0, note=62, velocity=80, time=0),
    Message('note_off', channel=0, note=62, velocity=0, time=256),
    Message('note_on', channel=0, note=60, velocity=80, time=0),
    Message('note_o

In [13]:
midi = mido.MidiFile('Datos/MIDIs/midi_data/F01_0006_0002_1.mid')
midi

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='é\x92¢ç\x90´, Piano', time=0),
    MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('set_tempo', tempo=555556, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=7, value=100, time=0),
    Message('control_change', channel=0, control=10, value=64, time=0),
    Message('control_change', channel=0, control=91, value=0, time=0),
    Message('control_change', channel=0, control=93, value=0, time=0),
    MetaMessage('midi_port', port=0, time=0),
    Message('note_on', channel=0, note=67, velocity=80, time=0),
    Message('note_on', channel=0, note=67, velocity=0, time=341),
    Message('note_on', channel=0, note=69, velocity

In [ ]:
micro = 341
micro_p_min = 6e+7
bpm = 108

(micro/micro_p_min) * bpm

In [ ]:
micro_p_min/555556